In [68]:
def sum_of_squares(list1):
    return sum([element ** 2 for element in list1])

In [69]:
print sum_of_squares([1,2,3,4,5])
print sum_of_squares([])

55
0


# Poker Game Program

Each card has a rank and suite. For 5J, rank is 5, suite is J. <br>

Write a program that picks the best ranked hand.  <br>
The following is the list of different kinds of hand in ascending order of importance/rank

<b>0) Nothing or High Card:</b> <br>
When two hands fall in this category, we break the tie based on the highest card of each hand<br><br>


<b>1) pair:</b> <br>
Two of a kind. Others are all of different kind.<br><br>

<b>2)Two pair:</b><br>
Two of a kind x. Two more of same kind y<br><br>


<b>3)3 of a kind:</b><br>
 three cards have same rank<br><br>

<b>4) straight:</b><br>
The ranks of the hand form a sequence.<br><br>

<b>5) Flush:</b><br>
All cards in the hand have same suite.<br><br>

<b>6) Full House:</b><br>
Three cards of one rank. Two cards have another rank.<br><br>

<b>7) Four of a kind:</b><br>
four cards have same rank<br><br>

<b>8) Straigh flush:</b><br>
Straight and Flush.<br><br>

In [70]:
def card_ranks(hand):
    "Returns ranks of hand in reverse order"
    special_cards = {"T": 10, "J": 11, "Q": 12, "K" : 13, "A" : 14}
    return sorted([ int(card[0]) if card[0] not in special_cards else special_cards[card[0]]   for card in hand ],reverse = True)

def straight(ranks):
    "Returns whether the given ranks form a straight"    
    for i in range(0,len(ranks)-1):
        if ranks[i] != ranks[i+1] + 1:
            return False
    return True

def flush(hand):
    "Returns True if all cards have same suite"
    return len(set([card[1] for card in hand])) == 1

from collections import Counter            
def kind(n, ranks):
    "Return the rank that repeats n times. When there is no such unique rank, this function return False"
    counter = Counter(ranks)
    ranks_with_freq_n = [k for k,v in counter.iteritems() if v == n ] 
    if len(ranks_with_freq_n) == 1:
        return ranks_with_freq_n[0]
    else:
        return None
        

def two_pair(ranks):
    "Return reverse sorted two_pair for given rank. Else return None"
    counter = Counter(ranks)
    ranks_with_freq_n = [k for k,v in counter.iteritems() if v == 2 ] 
    if len(ranks_with_freq_n) == 2:
        return sorted(ranks_with_freq_n, reverse= True)
    else:
        return None
    
    
def hand_rank(hand):
    "Return the rank of given hand: hand_rank(hand) => value"
    ranks = card_ranks(hand)
    # card_ranks returns the ranks in reverse sorted order.
    if straight(ranks) and flush(hand):
        return (8,max(ranks))
    elif kind(4, ranks):
        return (7, kind(4,ranks),kind(1,ranks))
    elif kind(3, ranks) and kind(2, ranks):
        return (6, kind(3, ranks), kind(2, ranks))
    elif flush(hand):
        return (5, max(ranks))
    elif straight(ranks):
        return (4, max(ranks))
    elif kind(3, ranks):
        # TODO should include the two other numbers in the tuple
        return (3, kind(3, ranks), ranks)
    elif two_pair(ranks):
        return(2, two_pair(ranks),ranks)
    elif kind(2,ranks):
        return (1, kind(2, ranks), ranks)
    else:
        return (0, ranks)

def poker(hands):
    "Return the best hand: poker([hand,...]) => hand"
    return max(hands, key = hand_rank)

In [71]:
def test():
    "Test cases for the functions in the poker program"
    #### TEST 1
    # straight flush
    sf = "6C 7C 8C 9C TC".split()
    # four ok a kinf
    fk = "9D 9H 9S 9C 7D".split()
    # full house
    fh = "TD TC TH 7C 7D".split()
    assert poker([sf,fk,fh]) == sf
    #### TEST 2
    assert poker([fk,fh]) == fk
    #### TEST 3
    assert poker([fh,fh]) == fh
    #### TEST 4
    assert poker([fh]) == fh
    #### TEST 5
    assert poker([sf] + 99*[fh]) == sf
    #### TEST CASES for hand_rank
    assert hand_rank(sf) == (8,10)
    assert hand_rank(fk) == (7,9,7)
    assert hand_rank(fh) == (6,10,7)
    return "tests pass"

In [72]:
print test()

tests pass
